In [1]:
pip install SpeechRecognition


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
!git init

!git config --global user.name "AyushXRane"  
!git config --global user.email "ayush.rane@gmail.com"  

!git remote add origin https://github.com/AyushXRane/Song-Player.git

# Step 4: Add all files to the staging area
!git add .

# Step 5: Commit your changes
!git commit -m "Add initial Jupyter notebook"

xcode-select: note: no developer tools were found at '/Applications/Xcode.app', requesting install. Choose an option in the dialog to download the command line developer tools.
xcode-select: note: no developer tools were found at '/Applications/Xcode.app', requesting install. Choose an option in the dialog to download the command line developer tools.
xcode-select: note: no developer tools were found at '/Applications/Xcode.app', requesting install. Choose an option in the dialog to download the command line developer tools.
xcode-select: note: no developer tools were found at '/Applications/Xcode.app', requesting install. Choose an option in the dialog to download the command line developer tools.
xcode-select: note: no developer tools were found at '/Applications/Xcode.app', requesting install. Choose an option in the dialog to download the command line developer tools.
xcode-select: note: no developer tools were found at '/Applications/Xcode.app', requesting install. Choose an optio

In [2]:
import sys
print(sys.version)

3.11.9 (v3.11.9:de54cf5be3, Apr  2 2024, 07:12:50) [Clang 13.0.0 (clang-1300.0.29.30)]


In [3]:
!pip install numpy



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


In [5]:
!pip install tensorflow==2.14


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


In [5]:
!pip install scipy
!pip install sounddevice


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


In [6]:
!pip install transformers


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


In [23]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import speech_recognition as sr


# Parameters
sample_rate = 16000  # Sampling rate in Hz
channels = 1  # Number of audio channels (1 = mono, 2 = stereo)
frame_duration = 0.5  # Frame duration in seconds
silence_threshold = 500  # Silence threshold in audio amplitude
max_silence_duration = 5  # Stop recording after this many seconds of silence

def is_silent(audio_chunk):
    """
    Check if the audio chunk is silent.
    """
    return np.max(np.abs(audio_chunk)) < silence_threshold

def record_audio():
    """
    Record audio dynamically, stopping when silence is detected.
    """
    audio_buffer = []
    silence_duration = 0
    
    print("Recording... Press Ctrl+C to stop manually.")
    
    # Open a stream
    with sd.InputStream(samplerate=sample_rate, channels=channels) as stream:
        while True:
            # Read audio chunk
            audio_chunk = stream.read(int(sample_rate * frame_duration))[0]
            audio_chunk = audio_chunk.flatten()  # Ensure 1D array
            
            # Add chunk to buffer
            audio_buffer.append(audio_chunk)
            
            # Check for silence
            if is_silent(audio_chunk):
                silence_duration += frame_duration
            else:
                silence_duration = 0  # Reset silence duration if sound is detected
            
            # Stop recording after max silence duration
            if silence_duration >= max_silence_duration:
                print("Silence detected. Stopping recording.")
                break
    
    # Combine all chunks into one array
    audio_data = np.concatenate(audio_buffer)
    return audio_data

def save_audio_to_wav(audio_data, filename="output.wav"):
    """
    Save the recorded audio to a WAV file.
    """
    wav.write(filename, sample_rate, (audio_data * 32767).astype(np.int16))
    print(f"Audio saved to {filename}")

def recognize_audio(audio_data):
    """
    Perform speech recognition on the recorded audio.
    """
    recognizer = sr.Recognizer()
    try:
        # Convert NumPy array to WAV format in memory
        wav_data = (audio_data * 32767).astype(np.int16).tobytes()
        audio = sr.AudioData(wav_data, sample_rate, 2)
        
        # Recognize speech
        print("Recognizing speech...")
        text = recognizer.recognize_google(audio)
        print("You said:", text)
        return text
    except sr.UnknownValueError:
        print("Could not understand the audio.")
    except sr.RequestError as e:
        print(f"Speech recognition error: {e}")

def analyze_sentiment(text):
    from transformers import pipeline
    emotion_analyzer = pipeline("text-classification", 
                                model="SamLowe/roberta-base-go_emotions", top_k=None, 
                                )
    emotion_score = emotion_analyzer(text)
    for emotion in emotion_score[0]:
        print(f"{emotion['label']}:{emotion['score']:.2f}")

    highest_emotion = max(emotion_score[0], key=lambda x: x['score'])
    return highest_emotion['label']

# Main execution
try:
    audio_data = record_audio()  # Record dynamically with silence detection
    save_audio_to_wav(audio_data, "recorded_audio.wav")  # Save the audio to a WAV file
    transcribed_text = recognize_audio(audio_data)  # Perform speech recognition
    
    print(f"Transcribed Text: {transcribed_text}")
    analyze_sentiment(transcribed_text)
    user_emotion = analyze_sentiment(transcribed_text)

except KeyboardInterrupt:
    print("\nRecording manually stopped.")


Recording... Press Ctrl+C to stop manually.
Silence detected. Stopping recording.
Audio saved to recorded_audio.wav
Recognizing speech...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You said: I just lost my dog hug I just lost my dog
Transcribed Text: I just lost my dog hug I just lost my dog


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
Device set to use 0


sadness:0.90
disappointment:0.08
grief:0.05
neutral:0.03
realization:0.02
remorse:0.02
admiration:0.01
annoyance:0.01
love:0.01
joy:0.01
approval:0.01
surprise:0.01
fear:0.01
nervousness:0.01
amusement:0.01
anger:0.01
caring:0.01
disgust:0.00
disapproval:0.00
gratitude:0.00
curiosity:0.00
desire:0.00
embarrassment:0.00
excitement:0.00
relief:0.00
confusion:0.00
optimism:0.00
pride:0.00


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
Device set to use 0


sadness:0.90
disappointment:0.08
grief:0.05
neutral:0.03
realization:0.02
remorse:0.02
admiration:0.01
annoyance:0.01
love:0.01
joy:0.01
approval:0.01
surprise:0.01
fear:0.01
nervousness:0.01
amusement:0.01
anger:0.01
caring:0.01
disgust:0.00
disapproval:0.00
gratitude:0.00
curiosity:0.00
desire:0.00
embarrassment:0.00
excitement:0.00
relief:0.00
confusion:0.00
optimism:0.00
pride:0.00


In [9]:
!pip install spotipy
import spotipy

import spotipy
from spotipy.oauth2 import SpotifyOAuth



sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope="user-modify-playback-state",
                                              client_id = "e96cdbe81c4245bfb67c1791dfad591f",
                                              client_secret = "97f58619512a4234a8e78929bc65ea08",
                                              redirect_uri = "http://localhost:8080/",
                                              cache_path=".spotify_cache" 
 ) )


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


In [10]:

  emotions = {
    "embarrassment": "uplifting confidence boost",
    "disappointment": "healing and hopeful tracks",
    "annoyance": "chill and calming vibes",
    "disgust": "detox your mind",
    "sadness": "mellow and comforting melodies",
    "anger": "calming and peaceful tunes",
    "neutral": "relaxed and easygoing tracks",
    "surprise": "adventurous and exciting sounds",
    "disapproval": "soft reflective songs",
    "fear": "soothing and reassuring vibes",
    "realization": "thoughtful and introspective beats",
    "remorse": "gentle and reflective ballads",
    "curiosity": "explorative and eclectic tracks",
    "confusion": "grounding and centering sounds",
    "approval": "upbeat and celebratory anthems",
    "nervousness": "relaxing and steady rhythms",
    "grief": "soothing and empathetic melodies",
    "admiration": "inspiring and epic tunes",
    "amusement": "fun and playful beats",
    "excitement": "energetic and dynamic tracks",
    "love": "romantic and heartfelt songs",
    "joy": "happy and upbeat anthems",
    "gratitude": "warm and uplifting melodies",
    "pride": "bold and empowering tracks",
    "relief": "light and peaceful tunes",
    "optimism": "positive and motivational beats",
    "caring": "gentle and nurturing vibes",
    "desire": "sultry and passionate tracks"
}



In [24]:
keyword = emotions.get(user_emotion, "pop")
result = sp.search(q = keyword, limit = 1, type = "playlist")
if result["playlists"]["items"]:
    result_uri = result["playlists"]["items"][0]["uri"]
    sp.start_playback(context_uri = result_uri)
    print(f"Playing playlist: {result['playlists']['items'][0]['name']}")
else:
    print("No playlists found for the given query.")


Playing playlist: Mellow '70s Gold
